# Molecules - Task 

* Classify the molecules of the validation set using kNN 
* Distance: approximate Graph Edit Distance (GED)


## Data 

We have .gxl files. Those are XMLs with a lot of information. 
→ Only use the chemical symbol node label and the unlabeled, undirected edges:
``` 
    <node id="_1">
        <attr name="symbol">
            <string>C</string>
        </attr>
    </node>
     <edge from="_1" to="_2">
     </edge> 
```


Real example: 
Node 1: 

```
<node id="_1">
    <attr name="symbol">
        <string>O  </string>
    </attr>
    <attr name="chem">
        <int>2</int>
    </attr>
    <attr name="charge">
        <int>0</int>
    </attr><attr name="x">
        <float>4.9921</float>
    </attr>
    <attr name="y">
        <float>1.6561
        </float>
    </attr>
</node>
```

Edges from Node 1: 

```
<edge from="_1" to="_2">
    <attr name="valence"> 
    <int>1</int>
    </attr>
</edge>
```

## Parse Data correctly 


In [32]:
import xml.etree.ElementTree as ET
import os 

def createLists(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    nodes = []
    edges = []
    for node in tree.findall('.//node'):
        for key, val in node.attrib.items():
            nodes.append(int(val.replace("_", "")))
    for edge in tree.findall('.//edge'):
        edges.append([int(edge.attrib['from'].replace('_', '')), int(edge.attrib['to'].replace('_', ''))])
    return nodes, edges

nodes_ =[]
edges_ = []
molecules = {}
for fn in os.scandir('../Molecules/MoleculesClassification/gxl/'):
    # Only include XML files
    if not fn.name.endswith('.gxl'): 
        continue

    molecules[fn.name] = {'nodes': [], 'edges' : []}
    molecules[fn.name]['nodes'].append(createLists(fn.path)[0])
    molecules[fn.name]['edges'].append(createLists(fn.path)[1])


### How to read the dictionary
In molecules all molecules with their nodes and their edges are saved. 

Example: Get all edges from molecule 31510.gxl: 


In [33]:
print(molecules['31510.gxl']['edges'])

[[[1, 2], [2, 3], [1, 5], [6, 7], [7, 8], [8, 9], [2, 10], [4, 11], [3, 12], [3, 13], [5, 14], [7, 15], [15, 16], [16, 17], [8, 18], [18, 19], [19, 20], [20, 21], [20, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [31, 32], [32, 33], [33, 34], [34, 35], [4, 5], [9, 10], [6, 10], [4, 13]]]
